In [ ]:
!pip install yfinance
!pip install ta

In [2]:
import pandas as pd, numpy as np
import yfinance as yf

from copy import copy
import statistics as stats
import math
from functools import reduce

from ta.volume import MFIIndicator
from ta.volatility import AverageTrueRange
from ta.trend import STCIndicator
from ta.trend import EMAIndicator

# Import des données historiques

In [3]:
ndx = yf.Ticker("^NDX")
df_historical_data = ndx.history(interval="1d", period="max")
df_historical_data.drop(columns=["Dividends","Stock Splits"], inplace=True)
df_historical_data.reset_index(inplace=True)

In [4]:
df_historical_data.describe()

,Open,High,Low,Close,Volume
count,9470.000000,9470.000000,9470.000000,9470.000000,9.470000e+03
mean,2931.704905,2956.226136,2905.688608,2932.321886,1.611191e+09
std,3556.703623,3584.502784,3526.220204,3557.270250,1.290133e+09
min,107.160004,108.269997,106.750000,107.160004,3.174000e+07
25%,424.767502,427.720001,423.322495,425.030006,4.721200e+08
50%,1638.229980,1657.420044,1624.675049,1640.315002,1.664305e+09
75%,3708.869934,3755.152466,3664.530029,3706.980042,2.069265e+09
max,16644.769531,16764.859375,16523.830078,16573.339844,1.162119e+10


In [5]:
# Test si aucune ligne manquante
test_list = [champ == 0 for champ in df_historical_data.isnull().sum()]

# Si toutes les colonnes sont True, résultat = True
res1 = all(i for i in test_list)
res1

True

In [6]:
df_historical_data.tail(1)

,Date,Open,High,Low,Close,Volume
9469,2023-04-27 00:00:00-04:00,12963.200195,13175.618164,12938.498047,13160.46582,971604285


In [7]:
df_historical_data.dtypes

Date      datetime64[ns, America/New_York]
Open                               float64
High                               float64
Low                                float64
Close                              float64
Volume                               int64
dtype: object

# Indicateurs techniques

### Alphatrend

In [8]:
# Trend indicator, équivalent de l'affichage couleur
def trend_indicator(trend):
    if trend > 0 :
        # Uptrend
        x = 1
    elif trend < 0 :
        # Downtrend
        x = -1
    else :
        # Range
        x = 0
    return x

In [9]:
# Defintion fonction
def generate_alphatrend(df_in, mfi_p, mfi_seuil, atr_l, m):
    '''Paramètres d'entrée : longueur MFI, longueur ATR, multiplier
    Retourne les colonnes Alphatrend, Alphatrend +2, Trend (position AT1 / AT2)
    :mfi_p = période MFI servant à délimiter up/down de l'alphatrend
    :mfi_seuil = période MFI pour recherche crossover, détermine uptrend ou downtrend'''

    df = df_in.copy()

    # Colonnes MFI
    s_mfi = MFIIndicator(high=df.High, low=df.Low, close=df.Close, volume=df.Volume, window=mfi_p).money_flow_index()
    df["MFI_ref"] = s_mfi

    # Colonne ATR
    s_atr = AverageTrueRange(high=df.High, low=df.Low, close=df.Close, window=atr_l).average_true_range()
    df["ATR"] = s_atr

    # Lignes UpT et DownT
    df["UpT_support"] = df["Low"] - df["ATR"] * m
    df["DownT_support"] = df["High"] + df["ATR"] * m

    # Suppression des lignes sans signal, en début de DataFrame
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    # ===============================================
    # Calcul Alphatrend, en tant que série
    
    Alphatrend = [0]

    for i in range (1, df.shape[0]):
        # Cas Uptrend
        if df.at[i,"MFI_ref"] >= mfi_seuil :
            if df.at[i,"UpT_support"] < Alphatrend[-1] :
                # Flat
                Alphatrend.append(Alphatrend[-1])
            else :
                # Trailing stop loss Up
                Alphatrend.append(df.at[i,"UpT_support"])

        # Cas Downtrend, MFI < 50
        else :
            if df.at[i,"DownT_support"] > Alphatrend[-1] :
                # Flat
                Alphatrend.append(Alphatrend[-1])
            else :
                # Trailing stop loss Down
                Alphatrend.append(df.at[i,"DownT_support"])

    # ===============================================
    # Ajout des lignes k1 et k2 en tant que colonnes
    
    if df.shape[0] == len(Alphatrend):
        df["Alphatrend_k1"] = pd.Series(Alphatrend).apply(lambda x: round(x,2))
        # Ligne k2 décalée de 2j
        Alphatrend2 = df["Alphatrend_k1"].shift(periods=2, fill_value=0)
        df["Alphatrend_k2"] = pd.Series(Alphatrend2).apply(lambda x: round(x,2))
        # Trend
        df["Trend"] = df.Alphatrend_k1 - df.Alphatrend_k2
        df["Trend"] = df["Trend"].apply(trend_indicator)
    else :
        print("Erreur lors de la génération des lignes Alphatrend")

    # ===============================================
    # Génération des signaux Achat / Vente

    # On isole tous les index non neutres, où AT1 != AT2, à la hausse (1) comme à la baisse (-1)
    s_trend = df["Trend"].loc[df["Trend"]!=0]
    s_trend_diff = s_trend - s_trend.shift(1)

    buy_signal_indexes = s_trend_diff[s_trend_diff == 2].index
    sell_signal_indexes = s_trend_diff[s_trend_diff == -2].index

    df["Signal"] = 0
    df.loc[buy_signal_indexes,"Signal"] = 1
    df.loc[sell_signal_indexes,"Signal"] = -1

    # ===============================================
    # Spécification des colonnes avec les paramètres d'entrée
    params = "MFIp = " + str(mfi_p) + ", MFItrigger = " + str(mfi_seuil) + ", ATR = " + str(atr_l) + ", m = " + str(m)
    # Sélection des colonnes suffisantes
    df = df[["Date","Alphatrend_k1","Alphatrend_k2","Trend","Signal"]]
    
    return df, params

In [10]:
df_AT, parametres_AT = generate_alphatrend(df_historical_data, mfi_p=14, mfi_seuil=50, atr_l=13, m=0.2)

In [11]:
# Servira à stocker les paramètres testés pour identifier la meilleure combinaison
print(parametres_AT)

MFIp = 14, MFItrigger = 50, ATR = 13, m = 0.2


In [12]:
df_AT.tail(1)

,Date,Alphatrend_k1,Alphatrend_k2,Trend,Signal
9456,2023-04-27 00:00:00-04:00,12968.72,12972.23,-1,0


### STC & EMA

In [13]:
def generate_STC_and_EMA(df_in, stc_length, fast_length, slow_length, ema_period):
  
  df = df_in[["Date","Close"]].copy()

  s_stc = STCIndicator(close=df.Close, window_slow=slow_length, window_fast=fast_length, cycle=stc_length).stc()
  s_ema = EMAIndicator(close=df.Close, window=ema_period).ema_indicator()

  df["STC"] = round(s_stc,2)
  df["EMA"] = round(s_ema,2)

  params = "STC Length = " + str(stc_length) + ", Fast Length = " + str(fast_length) + ", Slow Length = " + str(slow_length) + ", EMA length = " + str(ema_period)

  df.drop(columns=["Close"], inplace=True)

  return df, params

In [14]:
df_STC_EMA, params_STC_EMA = generate_STC_and_EMA(df_historical_data, stc_length=130, fast_length=25, slow_length=125, ema_period=70)

In [15]:
# Servira à stocker les paramètres testés pour identifier la meilleure combinaison
print(params_STC_EMA)

STC Length = 130, Fast Length = 25, Slow Length = 125, EMA length = 70


In [16]:
df_STC_EMA.tail(1)

,Date,STC,EMA
9469,2023-04-27 00:00:00-04:00,98.09,12506.37


### Merge et export du Dataset contenant l'ensemble des indicateurs techniques

In [63]:
df_essentials = df_historical_data.copy()

# Ajout de la colonne ATR qui servira plus tard dans le calcul de la sortie.
s_atr = AverageTrueRange(high=df_essentials.High, low=df_essentials.Low, close=df_essentials.Close, window=14).average_true_range()
df_essentials["ATR"] = s_atr

# Réduction au strict nécessaire pour les colonnes
df_essentials = df_essentials[["Date","Open","Close","ATR"]].copy()

df_essentials["Open"] = df_essentials.Open.apply(lambda x: round(x,2))
df_essentials["Close"] = df_essentials.Close.apply(lambda x: round(x,2))
df_essentials["ATR"] = df_essentials.ATR.apply(lambda x: round(x,2))

In [64]:
data_frames = [df_essentials, df_AT, df_STC_EMA]
df_IT = reduce(lambda  left,right: pd.merge(left,right, on=['Date'], how='left'), data_frames)

In [65]:
df_IT = df_IT.loc[df_IT["Date"]>="1998-01-01"]

In [66]:
df_IT.head(1)

,Date,Open,Close,ATR,Alphatrend_k1,Alphatrend_k2,Trend,Signal,STC,EMA
3099,1998-01-02 00:00:00-05:00,990.8,1008.23,24.18,959.34,959.34,0.0,0.0,0.21,1031.32


In [67]:
df_IT["Trend"].value_counts()

 0.0    3043
 1.0    2134
-1.0    1194
Name: Trend, dtype: int64

In [68]:
df_IT.reset_index(drop=True,inplace=True)

In [69]:
# Test si aucune ligne manquante
test_list = [champ == 0 for champ in df_historical_data.isnull().sum()]
test_list2 = [champ == 0 for champ in df_historical_data.isna().sum()]

# Si toutes les colonnes sont True, résultat = True
res1 = all(i for i in test_list)
res2 = all(i for i in test_list2)

if res1 and res2 :
  print("Ok pour Backtesting")
else :
  print("Anomalies détectées")

Ok pour Backtesting


In [71]:
# Export
#df_IT.to_csv("/content/drive/MyDrive/Colab Notebooks/sources/TradingView_strategies/AT_STC_EMA_indicateurs_optimised_short.csv", header=True, index=False)

# Backtesting

## Détection des entrées

Stratégie :<br>
<li>Entreée : Buy signal + Prix > EMA + STC < seuil(25)</li>
<li>Sortie : Sell signal + Prix < EMA + STC > seuil(75)</li>

In [72]:
stc_seuil_bas = 25
# Valeur 3 pour signaux d'entrée valides
df_IT["Buy_entry"] = np.sign(df_IT.Close - df_IT.EMA) + df_IT.Signal + np.sign(stc_seuil_bas - df_IT.STC)

stc_seuil_haut = 75
# Valeur -3 pour signaux d'entrée valides
# Attention / par deux signaux négatifs -> positif, d'où l'inversion sur un seul champ
df_IT["Sell_entry"] = np.sign(df_IT.Close - df_IT.EMA) + df_IT.Signal + np.sign(stc_seuil_haut - df_IT.STC)

In [73]:
# Conversion en np array
arr_buy_entry = df_IT["Buy_entry"].to_numpy()
# np.where(condition, vrai, sinon)
df_IT["Buy_entry"] = np.where(arr_buy_entry==3, 1, 0)

arr_sell_entry = df_IT["Sell_entry"].to_numpy()
df_IT["Sell_entry"] = np.where(arr_sell_entry==-3.0, -1, 0)

In [74]:
'''Agrégation dans une seule colonne "Entrées"
1 pour Buy -1 pour Sell
Ne sera peut-être pas conservé si paramètres différents entre Stratégie Short ou Long.
Il ne sera pas possible de générer les deux en même temps'''

df_IT["Entry"] = df_IT["Sell_entry"] + df_IT["Buy_entry"]
df_IT["Entry"].value_counts()

 0    6299
 1      50
-1      22
Name: Entry, dtype: int64

In [75]:
df_IT.drop(columns=["Buy_entry","Sell_entry"], inplace=True)
df_IT.tail(1)

,Date,Open,Close,ATR,Alphatrend_k1,Alphatrend_k2,Trend,Signal,STC,EMA,Entry
6370,2023-04-27 00:00:00-04:00,12963.2,13160.47,209.5,12968.72,12972.23,-1.0,0.0,98.09,12506.37,0


In [76]:
df_IT.loc[ df_IT["Entry"]==-1 ].index

Int64Index([ 781,  857,  869,  892,  908, 1244, 1272, 1942, 2804, 3493, 3511,
            3807, 3890, 4266, 4515, 4529, 5055, 5224, 5572, 6205, 6280, 6337],
           dtype='int64')

# Mesure de la performance

### Isolement des entrées

In [77]:
# Ajout d'une unité pour entrée le lendemain du signal confirmé et clos
short_entries_indexes = df_IT.loc[ df_IT["Entry"]==-1 ].index
short_entries_indexes += 1
short_entries_indexes = short_entries_indexes.to_list()
print(short_entries_indexes)

[782, 858, 870, 893, 909, 1245, 1273, 1943, 2805, 3494, 3512, 3808, 3891, 4267, 4516, 4530, 5056, 5225, 5573, 6206, 6281, 6338]


In [80]:
index_min = short_entries_indexes[0]
index_max = short_entries_indexes[-1]
print("Sélection des lignes entre {} et {}".format(index_min,index_max))

Sélection des lignes entre 782 et 6338


In [81]:
df_short_entries = df_IT.loc[index_min:index_max]
df_short_entries.shape

(5557, 11)

In [85]:
df_short_entries = df_short_entries[['Date','Open','Close','ATR']].copy()

### Conditions de sortie
2 ATR en stop Loss, 1:3 en Risk Reward

In [94]:
entree_reference = df_short_entries.at[782,"Open"]
atr_reference = df_short_entries.at[782,"ATR"]

stop_loss = entree_reference + 2 * atr_reference
take_profit = entree_reference - 3 * (2 * atr_reference)

print("Open : {}, Stop Loss : {}, Take Profit : {}".format(entree_reference,stop_loss,take_profit))

Open : 2436.48, Stop Loss : 2678.62, Take Profit : 1710.06


In [95]:
df_short_entries["Stop_Loss"] = 0
df_short_entries["Take_Profit"] = 0

In [88]:
df_short_entries.head()

,Date,Open,Close,ATR,Stop_Loss,Take_Profit
782,2001-02-08 00:00:00-05:00,2436.48,2355.67,121.07,2678.62,1710.06
783,2001-02-09 00:00:00-05:00,2333.01,2261.77,120.15,2573.31,1612.11
784,2001-02-12 00:00:00-05:00,2247.20,2286.76,118.04,2483.28,1538.96
785,2001-02-13 00:00:00-05:00,2315.46,2208.40,121.03,2557.52,1589.28
786,2001-02-14 00:00:00-05:00,2224.35,2305.82,121.95,2468.25,1492.65


In [96]:
df_short_entries.loc[782:857,"Stop_Loss"] = stop_loss
df_short_entries.loc[782:857,"Take_Profit"] = take_profit

In [97]:
df_short_entries

,Date,Open,Close,ATR,Stop_Loss,Take_Profit
782,2001-02-08 00:00:00-05:00,2436.48,2355.67,121.07,2678.62,1710.06
783,2001-02-09 00:00:00-05:00,2333.01,2261.77,120.15,2678.62,1710.06
784,2001-02-12 00:00:00-05:00,2247.20,2286.76,118.04,2678.62,1710.06
785,2001-02-13 00:00:00-05:00,2315.46,2208.40,121.03,2678.62,1710.06
786,2001-02-14 00:00:00-05:00,2224.35,2305.82,121.95,2678.62,1710.06
...,...,...,...,...,...,...
6334,2023-03-07 00:00:00-05:00,12303.19,12152.17,229.84,0.00,0.00
6335,2023-03-08 00:00:00-05:00,12181.06,12215.33,223.09,0.00,0.00
6336,2023-03-09 00:00:00-05:00,12230.84,11995.88,233.60,0.00,0.00
6337,2023-03-10 00:00:00-05:00,12001.70,11830.28,236.89,0.00,0.00


# Approche Turtle, sortie nb jours fixe

In [32]:
''' Isolement sous forme de série du lendemain de l'index détecté pour l'entrée.
Ici dans un range de +10j.
On en extrait la date, la variation min & max sur la période (sans préciser quand ils ont été atteints)'''
serie_10j = df_IT["Close"].iloc[120:130]
print("entrée à : {}, le {}".format(serie_10j[120], df_IT["Date"].iloc[120].strftime("%d/%m/%Y")) )
print("max : {}, soit Perf de {:.2f}%".format(serie_10j.max(), ((serie_10j.max()-serie_10j[120])/serie_10j[120])*100 ) )
print("max : ", serie_10j.min())

entrée à : 1311.4, le 25/06/1998
max : 1383.22, soit Perf de 5.48%
max :  1311.4
